In [ ]:
#関心領域のポリゴン情報の取得
from IPython.display import HTML
HTML(r'<iframe width="960" height="480" src="https://www.keene.edu/campus/maps/tool/" frameborder="0">')

In [ ]:
# 関心領域の座標情報を設定
AREA = [
      [
        139.0632368,
        35.1259218
      ],
      [
        139.0643526,
        35.1079486
      ],
      [
        139.0918184,
        35.106474
      ],
      [
        139.09302,
        35.1254304
      ],
      [
        139.0632368,
        35.1259218
      ]
    ]

In [ ]:
# 必要ライブラリのインポート
from glob import glob
from geojson import Polygon
import geopandas as gpd
import os
import json
import zipfile
import rasterio as rio
import rasterio.mask
from osgeo import gdal
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm
import shutil

In [ ]:
# 関心領域の地理情報ポリゴン作成
m = Polygon([AREA])
object_name = "atami"

with open(f"{object_name}.geojson", "w") as f:
  json.dump(m, f)

In [ ]:
# 作業領域の作成
sentinel_dir = "./work/ImageSentinel2/"
os.makedirs(sentinel_dir, exist_ok=True)

# RGBtifファイルの一時置き場
tif_dir = "./work/TifSentinel2/"
os.makedirs(tif_dir, exist_ok=True)

# マスクtifファイルの一時置き場
mask_dir = "./work/MaskSentinel2/"
os.makedirs(mask_dir, exist_ok=True)

# JPEGファイルの一時置き場
jpeg_dir = "./work/JpegSentinel2/"
os.makedirs(jpeg_dir, exist_ok=True)

# ZIPファイルの解凍処理
zip_list = glob("./images/*.zip")
for zip in tqdm(zip_list):
  with zipfile.ZipFile(zip) as zf:
    zf.extractall(sentinel_dir)

In [ ]:
# Sentinel-2データを読み込み、サンプルRGBファイルを作成する
def read_sentinel2(sentinel, rgb_path):
  path = sentinel + "/GRANULE/"
  files = os.listdir(path)
  pathA = path + str(files[0])
  files2 = os.listdir(pathA)
  pathB = pathA + "/IMG_DATA/R10m/"
  files3 = os.listdir(pathB)
  
  # Sentinel-2データの読み込み
  Red = rio.open(pathB + str(files3[0][0:23] + 'B04_10m.jp2'))
  Green = rio.open(pathB + str(files3[0][0:23] + 'B03_10m.jp2'))
  Blue = rio.open(pathB + str(files3[0][0:23] + 'B02_10m.jp2'))

  # サンプルRGBファイルの作成
  with rio.open(rgb_path, "w", driver="GTiff",
                width=Red.width, height=Red.height, count=3,
                crs=Red.crs, transform=Red.transform, dtype=Red.dtypes[0]) as rgb:
    rgb.write(Red.read(1), 1)
    rgb.write(Green.read(1), 2)
    rgb.write(Blue.read(1), 3)
    rgb.close() 
  
  return Red, Green, Blue

In [ ]:
def make_mask(input_geojson, rgb_path, band, mask_path):
  # ポリゴン情報の取得
  nReserve_geo = gpd.read_file(input_geojson)

  # 取得画像のEPSGを取得
  epsg = band.crs
  nReserve_proj = nReserve_geo.to_crs({"init": str(epsg)})

  # カラー合成画像より関心域を抽出
  with rio.open(rgb_path) as src:
    out_image, out_transform = rasterio.mask.mask(src, nReserve_proj.geometry, crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    with rasterio.open(mask_path, "w", **out_meta) as dest:
      dest.write(out_image)

In [ ]:
def make_jpeg(jpeg_path, mask_path):
  # 抽出画像のjpeg処理
  scale = '-scale 0 255 0 15'
  options_list = ['-ot Byte', '-of JPEG', scale]
  options_string = " ".join(options_list)

  # jpeg画像の保存
  gdal.Translate(jpeg_path, mask_path,
                 options=options_string)

In [ ]:
def add_sentinelDate(jpeg_path, date):
  img = Image.open(jpeg_path)

  # 日付の記載位置の設定
  x = img.size[0]/100
  y = img.size[1]/100

  # 画像に日付を記載
  obj_draw = ImageDraw.Draw(img)
  obj_draw.text((x, y), date, fill=(255, 255, 255))
  obj_draw.text((img.size[0]/2, img.size[1]-y-img.size[1]/20), 'produced fromESA remote sensing data', fill=(255, 255, 255))
  img.save(jpeg_path)


In [ ]:
sentinel_list = glob(f"{sentinel_dir}*")

for sentinel in tqdm(sentinel_list):
  # Sentinel-2の画像データを取得
  rgb_path = f"{tif_dir}{object_name}.tif"
  Red, Green, Blue = read_sentinel2(sentinel, rgb_path)

  # マスク画像の作成
  mask_path = f"{mask_dir}masked_{object_name}.tif"
  make_mask(f"{object_name}.geojson", rgb_path, Red, mask_path)
  
  # jpeg画像の作成
  date = sentinel.split("/")[3][11:19]
  jpeg_path = f"{jpeg_dir}{date}_masked_{object_name}.jpg"
  make_jpeg(jpeg_path, mask_path)
  

  # 画像への撮像日の記載
  add_sentinelDate(jpeg_path, date)  

In [ ]:
## 関心領域のGIFアニメーションを作成 ##
images = []
files = sorted(glob(f"{jpeg_dir}/*.jpg"))
images = list(map(lambda file: Image.open(file), files))

images[0].save(f"./{object_name}.gif", save_all=True, append_images=images[1:], duration=1000, loop=0)
